In [1]:
#Import the same libraries from our first script
import requests
from bs4 import BeautifulSoup
import pandas as pd

#only for Python2
import unicodecsv

#if you're using Python3 try this instead:
#import csv

In [2]:
#create a base_url that is just the domain and the directories that don't change
base_url = 'https://s3.amazonaws.com/python-at-ire15/death_row/'

In [3]:
#create a new_page variable with the html page name
new_page = 'dr_offenders_on_dr.html'

In [4]:
#concatenate the base_url and new_page into a url variable
url = base_url + new_page
print(url)

https://s3.amazonaws.com/python-at-ire15/death_row/dr_offenders_on_dr.html


In [5]:
#make a get request
r = requests.get(url)

In [6]:
#assign the r.content to html
#use the inspector and Developer Tools to see the HTML you need
html = r.content

In [7]:
#make a soup out of the html
soup = BeautifulSoup(html)

/Users/tommeagher/.virtualenvs/pandas/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 162 of the file /System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [8]:
#grab the body text. This is all the same as before
body_text = soup.find("div", id='body')

In [9]:
#pluck out the table from the body_text and assign it to "our_table"
our_table = body_text.find('table')

In [10]:
#find our first tr and assign it to headers
headers = our_table.find('tr')

In [11]:
#make an empty list that will hold our header names
myheaders = []

In [12]:
#make a for loop and append all the text of all the 'th' tags to the myheaders list
for th in headers.find_all('th'):
    myheaders.append(th.text)

In [13]:
#print myheaders out, so you can see how they differ
print(myheaders)

[u'TDCJ Number', u'Link', u'Last Name', u'First Name', u'Date of Birth', u'Gender', u'Race', u'Date Received', u'County', u'Date of Offense']


In [14]:
#create an output file name
output_file = 'deathrow.csv'

In [15]:
#open the f file object
with open(output_file,'wb') as f:
#for Python3, try this
#with open(output_file,'w', encoding='utf-8') as f:

    #create a w writer object
    w = unicodecsv.writer(f, encoding='utf-8')

    #for Python3, try this
    #w = csv.writer(f)

    #write our header rows to the csv using w
    w.writerow(myheaders)

    #grab all the trs after the first in our_table and loop through them
    #rename the variables in the right order and add new ones for offensedate, for instance
    for tr in our_table.find_all('tr')[1:]:
        tds = tr.find_all('td')
        number = tds[0].text
        link = tds[1].find('a').get('href')
        #have the class do the rest of these on their own
        last = tds[2].text
        first = tds[3].text
        dob = tds[4].text
        gender = tds[5].text
        race = tds[6].text
        indate = tds[7].text
        county = tds[8].text
        offensedate = tds[9].text
        #gather all the variables into a list
        rowitems = [number, link, last, first, dob, gender, race, indate, county, offensedate]
        #write the list to the csv with w
        w.writerow(rowitems)
#at the end of the loop, it automatically closes the f file

In [16]:
#read your local csv back into a pandas dataframe
deathrow = pd.read_csv('deathrow.csv')

In [17]:
#look at the top of your dataframe. Now you're ready for analysis
deathrow.head()

,TDCJ Number,Link,Last Name,First Name,Date of Birth,Gender,Race,Date Received,County,Date of Offense
0,999598,dr_info/williamseric.html,Williams,Eric,04/07/1967,M,White,12/18/2014,Kaufman,03/30/2013
1,999597,dr_info/thomassteven.html,Thomas,Steven,09/21/1958,M,White,12/05/2014,Williamson,11/04/1980
2,999596,dr_info/sunigabrian.html,Suniga,Brian,12/27/1979,M,White,10/30/2014,Lubbock,12/26/2011
3,999595,dr_info/lewisharlem.html,"Lewis, III",Harlem,07/05/1991,M,Black,08/01/2014,Harris,12/24/2012
4,999594,dr_info/valdezfidencio.html,Valdez,Fidencio,05/30/1979,M,Hispanic,07/24/2014,El Paso,12/10/2010
